**EXCEL'DEN ÖRNEK VERİYİ OKUMA VE PREPROCESSING ADIMLARI**

Şikayetvar.com Turkcell başlığından çekilen 10 text verisi ilk önce ön-işlemeden geçirilmiş ve modelde denenemesi için df'e çevrilmiştir.

PRE-TRAINED modeller kullanılmıştır, önceden eğitilmiş modeller kullanıldığı için modelin sonuçlarını hızlıca gözlemleyebilmek adına ilk adımda 10 veri ile deneme yapılmmıştır.

In [15]:
import pandas as pd

# Excel dosyasının yolunu belirtin
excel_dosyasi = "/content/dnm_önişlemli_trckcll_10tane.xlsx"

# Excel dosyasını okuyup DataFrame'e dönüştürmek
df = pd.read_excel(excel_dosyasi)

# DataFrame'i görüntülemek için
print(df.head())  # İlk birkaç satırı görmek için
df

                                            Metinler
0  turkcell superonline firması taşındığım yere i...
1  turkcell ’ in star 10 gb paketini kullanmaktay...
2  turkcell kullanmadığım aktif olmayan eski hatt...
3  turkcell ev internetini 1 yıldır doğru dürüst ...
4  turkcell internet paketim kendiliğinden bitiyo...


,Metinler
0,turkcell superonline firması taşındığım yere i...
1,turkcell ’ in star 10 gb paketini kullanmaktay...
2,turkcell kullanmadığım aktif olmayan eski hatt...
3,turkcell ev internetini 1 yıldır doğru dürüst ...
4,turkcell internet paketim kendiliğinden bitiyo...
5,2024 şubat ayında almış olduğum askercell hatt...
6,temmuz ayında hattıma tanımlanan yurt dışı pak...
7,gnç uygulamasından çatlat yaptım 9 saatlik int...
8,turkcell hattımın 2 yıllık taahhüdü şükür bitt...


In [3]:
!pip install TurkishStemmer
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [16]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from TurkishStemmer import TurkishStemmer
from transformers import pipeline

# NLTK Türkçe stopwords listesi (önceden indirmeniz gerekebilir)
stop_words = set(stopwords.words('turkish'))

# Türkçe kök bulucu
stemmer = TurkishStemmer()


In [17]:

def preprocess_text(text):
    # Küçük harfe dönüştürme
    text = text.lower()

    # Noktalama işaretleri ve özel karakterlerin kaldırılması
    text = re.sub(r'[^\w\s]', '', text)

    # Sayıların kaldırılması
    text = re.sub(r'\d+', '', text)

    # Stop words (durdurma kelimeleri) kaldırma
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    # Kök bulma (stemming)
    tokens = [stemmer.stem(word) for word in tokens]

    # Kelimeleri tekrar birleştirme
    return ' '.join(tokens)

# DataFrame'inizi burada yükleyin
# df = pd.read_csv('path_to_your_file.csv')  # Örneğin CSV'den yükleme


# Metinlere ön işleme adımlarını uygulama
df['cleaned_text'] = df['Metinler'].apply(preprocess_text)
print(df)

                                            Metinler  \
0  turkcell superonline firması taşındığım yere i...   
1  turkcell ’ in star 10 gb paketini kullanmaktay...   
2  turkcell kullanmadığım aktif olmayan eski hatt...   
3  turkcell ev internetini 1 yıldır doğru dürüst ...   
4  turkcell internet paketim kendiliğinden bitiyo...   
5  2024 şubat ayında almış olduğum askercell hatt...   
6  temmuz ayında hattıma tanımlanan yurt dışı pak...   
7  gnç uygulamasından çatlat yaptım 9 saatlik int...   
8  turkcell hattımın 2 yıllık taahhüdü şükür bitt...   

                                        cleaned_text  
0  turkcell superonlin firma taşındık yer interne...  
1  turkcell in star gb paket kullanmak paket içer...  
2  turkcell kullanmadık aktif olmayan esk hatt sü...  
3  turkcell ev internet yıl doğr dürüst kullanamı...  
4  turkcell internet pake kendilik bitiyor telefo...  
5  şubat ayın al olduk askercell hatt mart ayın a...  
6  temmuz ayın hatt tanımlanan yurt dış pake yurt...  

In [18]:
# storing into the excel file
df.to_excel("işlenmiş_10luk_dnm.xlsx")

**MODEL**

SONUÇLAR DOSYA KLASÖRÜ YENİLENDİĞİNDE analysis_results.json DOSYASINDA GÖZÜKECEKTİR. DOSYAYA ÇİFT TIKLAYIP AÇABİLİRSİNİZ.

In [23]:
import re
import pandas as pd
import json
from transformers import pipeline

# NER ve Sentiment modelini yükleme
ner_model = pipeline("ner", model="savasy/bert-base-turkish-ner-cased")
sentiment_model = pipeline("sentiment-analysis", model="savasy/bert-base-turkish-sentiment-cased")

def combine_subwords(entities):
    combined_entities = []
    current_entity = ""

    for entity in entities:
        # Sadece ORGANISATION etiketli entiteleri al
        if entity['entity'].startswith('B-') or entity['entity'].startswith('I-'):
            if entity['entity'] in ['B-ORG', 'I-ORG']:
                if entity['word'].startswith('##'):
                    current_entity += entity['word'].replace('##', '')
                else:
                    if current_entity:
                        combined_entities.append(current_entity)
                        current_entity = ""
                    current_entity = entity['word']

    if current_entity:
        combined_entities.append(current_entity)

    # Sonuçları anlamlı kelimeler olarak düzenle
    combined_entities = list(set(combined_entities))  # Duplicates might be removed here

    return combined_entities

def analyze_text(text):
    # Entity extraction
    entities = ner_model(text)

    # Subword tokenlerini birleştir ve sadece ORGANISATION entitelerini al
    combined_entities = combine_subwords(entities)

    # Her bir entity için duygu analizi
    results = []
    for entity in combined_entities:
        sentiment = sentiment_model(entity)
        results.append({
            "entity": entity,
            "sentiment": sentiment[0]['label']
        })

    return format_analysis(text, results)

def format_analysis(text, analysis):
    # Create entity list and results in desired format
    entity_list = list(set([item['entity'] for item in analysis if len(item['entity']) > 2]))
    results = [{'entity': item['entity'], 'sentiment': item['sentiment']} for item in analysis if len(item['entity']) > 2]
    return {
        'text': text,
        #'entity_list': entity_list,
        'results': results
    }

def save_to_json(df, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            json.dump(row['analysis'], f, ensure_ascii=False, indent=4)
            f.write('\n')


# DataFrame'deki metinler için analiz yapma
df['analysis'] = df['cleaned_text'].apply(analyze_text)

# Sonuçları bir JSON dosyasına kaydetme
save_to_json(df, 'analysis_results.json')




Some weights of the model checkpoint at savasy/bert-base-turkish-ner-cased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


![image.png]()